In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.


import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'predicting-employee-attrition:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1736414%2F2838014%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240314%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240314T135425Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dbe8eb4c456e932c04a03915054add39d5ad0373496f2c6e4d915d0640dabea4c6276a9e576f5b298fa6d5b24abecd3fe699a8776a0df1d0ae542de138501eaaa683e4b5b42ded90781ad647d9cad90a5402bf4d1b84cbbe6e1a21f3aa56de2c56057f0577f2f33a2c9253c81a6beaaf40e038b75ac7237b4c196b8a30fb72709839c2c9dde711bad86c81afa5ddb95990fd9913f197c5ccf3e1a92337185a6668d92eef38107e87fd14b17881e622ee0f494e22d43042920c103f310bb8e51eadd5ae6ae49c4a7cac06bfbed52277fafb18942916724d05398184b85731c09524822a9a4eae3e6a71838651de162cf73f3a324c85d7e2fc29bf630cbb2269acc'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Import Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/predicting-employee-attrition/train_data.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train['MMM-YY'].describe()

In [ ]:
train['MMM-YY'] = pd.to_datetime(train['MMM-YY'])
train['Dateofjoining'] = pd.to_datetime(train['Dateofjoining'])
train['LastWorkingDate'] = pd.to_datetime(train['LastWorkingDate'])

# Feature Engineering

In [ ]:
uniqueemployeesdata= train[['Emp_ID', 'Age', 'Gender', 'City', 'Education_Level',
       'Salary', 'Dateofjoining', 'Joining Designation','Designation']].drop_duplicates()

In [ ]:
# get the target column set
employeeswholeft = train[~train['LastWorkingDate'].isna()]
employeeswholeft['Y']=0
uniqueemployeesdata = uniqueemployeesdata.merge(employeeswholeft[['Emp_ID','Y']],on='Emp_ID',how='left')
uniqueemployeesdata['Y']=uniqueemployeesdata['Y'].fillna(1)

In [ ]:
employeeage = train.groupby('Emp_ID').count()['City']
employeeage = employeeage.reset_index()
employeeage.columns = ['Emp_ID','Periodwithus']
uniqueemployeesdata=  uniqueemployeesdata.merge(employeeage,on='Emp_ID',how='left')

In [ ]:
uniqueemployeesdata = uniqueemployeesdata.merge(pd.pivot_table(train,index='Emp_ID',values=['Total Business Value','Quarterly Rating'],aggfunc='mean').reset_index(),on='Emp_ID',how='left')

In [ ]:
# time business done
uniqueemployeesdata['timesbusinessdone'] = uniqueemployeesdata['Total Business Value']/uniqueemployeesdata['Salary']

In [ ]:
# reversal happend for employees
reversals = train[train['Total Business Value']<0]
reversals['reversal']=1
uniqueemployeesdata=uniqueemployeesdata.merge(reversals[['Emp_ID','reversal']].drop_duplicates(),on='Emp_ID',how='left')
uniqueemployeesdata['reversal']=uniqueemployeesdata['reversal'].fillna(0)

In [ ]:
# zero business count
zeroperformance = train[train['Total Business Value']==0]
zeroperformance = zeroperformance.groupby('Emp_ID').count()['Age'].reset_index()
zeroperformance.columns = ['Emp_ID','zeroperformance']
uniqueemployeesdata = uniqueemployeesdata.merge(zeroperformance.drop_duplicates(),on='Emp_ID',how='left')

In [ ]:
# months where coverage less than salary
train['timesbusiness'] = train['Total Business Value']/train['Salary']
businesslesscoverage = train[train['timesbusiness']<1]
businesslesscoverage=businesslesscoverage.groupby('Emp_ID').count()['Age'].reset_index()
businesslesscoverage.columns=['Emp_ID','Businesslessthansalary']
uniqueemployeesdata=uniqueemployeesdata.merge(businesslesscoverage[['Emp_ID','Businesslessthansalary']].drop_duplicates(),on='Emp_ID',how='left')
uniqueemployeesdata['Businesslessthansalary']=uniqueemployeesdata['Businesslessthansalary'].fillna(0)

In [ ]:
# Change in designation
changedesignation = train[train['Joining Designation'] != train['Designation']]
changedesignation['changedesignation']=1
changedesignation = changedesignation[['Emp_ID','changedesignation']].drop_duplicates()
uniqueemployeesdata = uniqueemployeesdata.merge(changedesignation[['Emp_ID','changedesignation']].drop_duplicates(),on='Emp_ID',how='left')
uniqueemployeesdata['changedesignation'] = uniqueemployeesdata['changedesignation'].fillna(0)

In [ ]:
# Last rating before leaving
lastrating = train[['Emp_ID','Quarterly Rating']].drop_duplicates( subset='Emp_ID',keep='last')
lastrating.columns = ['Emp_ID','Last Rating']
uniqueemployeesdata = uniqueemployeesdata.merge(lastrating,on='Emp_ID',how='left')

In [ ]:
uniqueemployeesdata.head()

In [ ]:
plt.figure(figsize=(15,7.5))
uniqueemployeesdata=uniqueemployeesdata[['Emp_ID', 'Age', 'City', 'Education_Level', 'Salary', 'Dateofjoining',
       'Joining Designation', 'Designation', 'Quarterly Rating',
       'Total Business Value', 'Gender',
       'timesbusinessdone','reversal','zeroperformance','Businesslessthansalary','changedesignation','Last Rating','Periodwithus','Y']]
df4_corr=uniqueemployeesdata.corr()
sns.heatmap(df4_corr,annot=True)

In [ ]:
!pip install pycaret

# Model Evaluation

In [ ]:
from pycaret.classification import *
s = setup(uniqueemployeesdata, target = 'Y', ignore_features = ['Emp_ID','Dateofjoining'])

In [ ]:
best_model = compare_models(sort='AUC')

In [ ]:
best_model

In [ ]:
tuned_best_model = tune_model(best_model)

In [ ]:
# performance of coverage
plot_model(tuned_best_model, plot = 'auc')

In [ ]:
# feature importance
plot_model(tuned_best_model, plot = 'feature')

In [ ]:
# confusion matrix
plot_model(tuned_best_model, plot = 'confusion_matrix')

In [ ]:
test = pd.read_csv('/kaggle/input/predicting-employee-attrition/test_data.csv')

In [ ]:
testset = uniqueemployeesdata[uniqueemployeesdata['Emp_ID'].isin(test['Emp_ID'])]
testset= testset.drop_duplicates()

In [ ]:
et = create_model('et')
tuned_et = tune_model(et)
predict_model(tuned_et)

In [ ]:
final_et = finalize_model(tuned_et)
unseen_predictions = predict_model(final_et, data=testset)
predictionsfinal= unseen_predictions[['Emp_ID','Label']].drop_duplicates()

In [ ]:
predictionsfinal.head()

In [ ]:
predictionsfinal = predictionsfinal.drop_duplicates( subset='Emp_ID',keep='last')
predictionsfinal.to_csv('predictions.csv')

Upvote, If you like the work. Thank you.